In [1]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard
from keras_tuner import HyperParameters, Hyperband
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from datetime import datetime
import pandas as pd, numpy as np, tensorflow as tf, os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from math import cos, pi
from tqdm.notebook import tqdm

2024-05-29 14:34:24.500447: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 14:34:24.504070: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 14:34:24.553063: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 14:34:25.346552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [31]:
hp = HyperParameters()
hp.Int('n_layers', min_value = 4, max_value = 15)
hp.Int('n_units', min_value = 32, max_value = 512, step = 32)
hp.Int('epochs', min_value = 50, max_value = 1000, step = 50)

hp.Float('penalty', min_value = 1e-4, max_value = 1, sampling = 'log')

hp.Choice('activation', ['relu', 'elu', 'softmax'])
hp.Choice('optimizer', ['adam', 'rmsprop', 'adamax'])

'adam'

In [2]:
def transformedWindDirection(cc):
    X, Y, d0 = cc
    d0 = 270 - d0
    inlandX, inlandY = 520000, 485000
    c = np.arctan2(Y - inlandY, X - inlandX) * 180/ pi
    # Wind direction relative to the direction from the center of Iceland to station
    # Wind directly from the ocean gives twd = 180
    twd = abs(d0 - c)
    if twd > 180:
        twd =  360 - twd
    return cos(twd * pi / 180)

In [17]:
def get_data_chunks():
    chunks_path = './data/chunks/'
    chunk_files = [chunks_path + chunk for chunk in os.listdir(chunks_path) if chunk.endswith('.feather')]
    df = pd.DataFrame()
    for file in tqdm(chunk_files, total = len(chunk_files), desc = 'Looping through chunks...'):
        c_df = pd.read_feather(file)
        c_df['gust_factor'] = c_df.fg / c_df.f
        df = pd.concat([df, c_df])
    return df

In [3]:
def unfoldElevations(df):
    elevations_array = np.array(df.elevations.tolist())
    elevation_columns = pd.DataFrame(elevations_array, index=df.index)
    elevation_columns.columns = [f'elevation_{i}' for i in range(elevations_array.shape[1])]
    df = df.drop(['elevations'], axis = 1)
    df = pd.concat([df, elevation_columns], axis=1)

    return df

In [5]:
def get_normalized_data_chunks():
    scaler = StandardScaler()
    df = get_data_chunks()
    df = df[df.gust_factor >= 1]
    print("Already filtered by gust factor being less than 1")
    df = df.reset_index()
    df = df.drop(['index'], axis = 1)
    tqdm.pandas(desc = 'Adding transformed wind direction...')
    df['cc'] = list(zip(df.X, df.Y, df.wd_15))
    df['twd'] = df.cc.progress_map(transformedWindDirection)
    df = df.drop(['cc', 'X', 'Y'], axis = 1)
    print("Finished adding transformed wind direction")
    pre_len = len(df.columns)
    df = unfoldElevations(df)
    n_components = len(df.columns) - pre_len
    df.iloc[:, -n_components:] = df.iloc[:, -n_components:].sub(df.station_elevation, axis = 0)
    print("Subtracted station elevation from landscape elevation")
    df = df.replace([-np.inf, np.inf], np.nan)
    df = df.dropna()
    df.columns = df.columns.astype(str)
    print("About to write df to feather...")
    df = df.reset_index()
    df.to_feather("./model_data.feather")
    print("Finished writing df to feather...")
    print("Ready to split")
    y = df.gust_factor
    X = df.drop(['gust_factor'], axis = 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
    X_train, X_val, X_test  = scaler.fit_transform(X_train), scaler.fit_transform(X_val), scaler.fit_transform(X_test)
    print("Train, val, test is ready")

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [11]:
def get_normalized_data():
    df = pd.read_feather("./data/all_data.feather")
    df = df[df.f >= 5]
    df = df.drop(['lon_x', 'lat_x', 'fx', 'lon_y', 'lat_y', 'LonLat'], axis = 1)
    df = df[df.gust_factor >= 1]
    print("Already filtered by gust factor being less than 1")
    df = df.reset_index()
    df = df.drop(['index'], axis = 1)
    tqdm.pandas(desc = 'Adding transformed wind direction...')
    df['cc'] = list(zip(df.X, df.Y, df.wd_15))
    df['twd'] = df.cc.progress_map(transformedWindDirection)
    df = df.drop(['cc', 'X', 'Y'], axis = 1)
    print("Finished adding transformed wind direction")
    pre_len = len(df.columns)
    df = unfoldElevations(df)
    n_components = len(df.columns) - pre_len
    df.iloc[:, -n_components:] = df.iloc[:, -n_components:].sub(df.station_elevation, axis = 0)
    print("Subtracted station elevation from landscape elevation")
    df = df.replace([-np.inf, np.inf], np.nan)
    df = df.dropna()
    df.columns = df.columns.astype(str)
    print("About to write df to feather...")
    df = df.reset_index().drop(['index'], axis = 1)
    df.to_feather("./data/model_data_with_time_station_measurements.feather")
    print("Finished writing df to feather...")
    return
    print("Ready to split")
    y = df.gust_factor
    X = df.drop(['gust_factor'], axis = 1)

    scaler = StandardScaler()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
    X_train, X_val, X_test  = scaler.fit_transform(X_train), scaler.fit_transform(X_val), scaler.fit_transform(X_test)
    print("Train, val, test is ready")

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [ ]:
def sortingByInterval():
    #Just add to get normalized data from file
    return
    df = df[df.f >= 15]
    df = df.sort_values(by = ['stod', 'time'])
    stations = df.stod.unique()
    threshold = '1 day'
    combined_df = pd.DataFrame()
    for station in tqdm(stations, total = len(stations), desc = "Looping over substations..."):
        subset_df = df[station == df.stod]
        subset_df = subset_df.reset_index(drop = True)
    
        columns, filtered_data = subset_df.columns, []
    
        while not subset_df.empty:
            idx = subset_df.f.idxmax()
            time_of_max = subset_df.iloc[idx].time
            filtered_data.append(subset_df.iloc[idx])
            subset_df = subset_df[abs(subset_df.time - time_of_max) >= pd.Timedelta(threshold)]
            subset_df = subset_df.reset_index(drop = True)
        filtered_df = pd.DataFrame(filtered_data, columns=columns)
        combined_df = pd.concat([combined_df, filtered_df], ignore_index = True)
    combined_df = combined_df.sort_values(by=['stod', 'time'])
    combined_df = combined_df.reset_index(drop = True)
    combined_df.to_feather("./data/filtered_by_interval_15ms_model_data.feather")

In [2]:
def get_normalized_data_from_file(file_path = "./data/filtered_by_interval_15ms_model_data.feather"): 
    #"./data/model_data_with_time_station_measurements.feather"):
    df = pd.read_feather(file_path)
    mean_gust_factor = df.gust_factor.mean()
    df = df.drop(['d', 'f', 'fg', 'stod', 'time'], axis = 1)
    y = df.gust_factor
    X = df.drop(['gust_factor'], axis = 1)

    scaler = StandardScaler()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
    X_train, X_val, X_test  = scaler.fit_transform(X_train), scaler.fit_transform(X_val), scaler.fit_transform(X_test)
    print("Train, val, test is ready")

    return mean_gust_factor, (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [5]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [8]:
def getData(model_data_path = "./model_data.feather"):
  df = pd.read_feather(model_data_path)
  scaler = StandardScaler()
  y = df.gust_factor
  X = df.drop(['gust_factor'], axis = 1)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
  X_train, X_val, X_test  = scaler.fit_transform(X_train), scaler.fit_transform(X_val), scaler.fit_transform(X_test)
  return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [9]:
def getDataFromDF(df):
  scaler = StandardScaler()
  y = df.gust_factor
  X = df.drop(['gust_factor'], axis = 1)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
  X_train, X_val, X_test  = scaler.fit_transform(X_train), scaler.fit_transform(X_val), scaler.fit_transform(X_test)
  return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [3]:
mean_gust_factor, train, val, test = get_normalized_data_from_file()

X_train, y_train = train
X_val, y_val = val
X_test, y_test = test

Train, val, test is ready


In [4]:
X_train.shape

(96319, 736)

In [28]:
def build_model(n_units = 256, activation = 'elu', penalty =  0.01, n_layers = 11, optimizer = 'rmsprop'): #penalty =  0.00168, n_units = 64
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape = (X_train.shape[1], )))
    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.BatchNormalization())

    for _ in range(n_layers):
        model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))

    model.compile(optimizer = optimizer, loss = mean_absolute_percentage_error)

    return model

In [33]:
def build_model_tuner(hp):
    n_units = hp.get('n_units')
    n_layers = hp.get('n_layers')
    activation = hp.get('activation')
    penalty = hp.get('penalty')
    optimizer = hp.get('optimizer')

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape = (X_train.shape[1], )))
    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.BatchNormalization())

    for _ in range(n_layers):
        model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))

    model.compile(optimizer = optimizer, loss = mean_absolute_percentage_error)

    return model

In [15]:
model = build_model(n_units = 160, activation = 'relu', penalty =  0.00012506, n_layers = 4, optimizer = 'adamax')

In [11]:
def baseline(y_test, mean_gust_factor):
    return mean_absolute_percentage_error(y_test, mean_gust_factor)

In [15]:
f'The baseline model error is {baseline(y_test, mean_gust_factor)}'

'The baseline model error is 9.651663519912155'

In [36]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True, verbose = 1)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
tuner = Hyperband(build_model_tuner, hyperparameters = hp, objective = 'val_loss', max_epochs = 1000, project_name = "try-2024-05-29-with-interval")
tuner.search(X_train, y_train, validation_data = (X_val, y_val), callbacks = [tensorboard_callback, early_stopping])

Trial 33 Complete [00h 00m 36s]
val_loss: 8.696076393127441

Best val_loss So Far: 7.377780437469482
Total elapsed time: 00h 24m 12s

Search: Running Trial #34

Value             |Best Value So Far |Hyperparameter
14                |10                |n_layers
352               |128               |n_units
700               |450               |epochs
0.083174          |0.0001275         |penalty
relu              |elu               |activation
rmsprop           |adamax            |optimizer
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
6                 |6                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


KeyboardInterrupt: 